In [ ]:
from __future__ import annotations

import typing as t
from enum import Enum
from pathlib import Path

from aiida import load_profile, orm
from aiida_quantumespresso.calculations.pp import PpCalculation
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_quantumespresso.workflows.pw.relax import PwRelaxWorkChain
from aiida_workgraph import dynamic, namespace, shelljob, task
from ase import Atoms

_ = load_profile()

In [ ]:
class AfmCase(Enum):
    EMPIRICAL = 1
    HARTREE = 2
    HARTREE_RHO = 3


@task
def write_afm_params(params: dict) -> orm.SinglefileData:
    afm_filepath = Path.cwd() / "params.ini"
    with open(afm_filepath, "w") as config_file:
        for key, value in params.items():
            if isinstance(value, (list, tuple)):
                value = " ".join(map(str, value))
            config_file.write(f"{key} {value}\n")
    return orm.SinglefileData(file=afm_filepath.as_posix())


@task
def write_structure_file(structure: Atoms, filename: str) -> orm.SinglefileData:
    geom_filepath = Path.cwd() / filename
    structure.write(geom_filepath, format="xyz")
    return orm.SinglefileData(file=geom_filepath.as_posix())


RelaxJob = task(PwRelaxWorkChain)
ScfJob = task(PwBaseWorkChain)
PpJob = task(PpCalculation)

In [ ]:
@task.graph
def AfmWorkflow(
    case: AfmCase,
    structure: orm.StructureData,
    afm_params: dict,
    relax: bool = False,
    dft_params: t.Annotated[
        dict[str, dict],
        namespace(
            structure=RelaxJob.inputs,
            tip=ScfJob.inputs,
        ),
    ] = None,
    pp_params: t.Annotated[
        dict[str, dict],
        namespace(
            hartree=PpJob.inputs,
            charge=PpJob.inputs,
        ),
    ] = None,
    tip: orm.StructureData = None,
) -> t.Annotated[dict, dynamic(t.Any)]:
    """AFM simulation workflow."""

    if relax:
        assert dft_params, "Missing DFT parameters"
        geom_dft_params = dft_params.get("structure", {})
        dft_job = RelaxJob(
            structure=structure,
            **geom_dft_params,
        )
        structure = dft_job.output_structure
    else:
        assert structure, "Missing structure"

    geometry_file = write_structure_file(structure, "geo.xyz").result

    assert afm_params, "Missing AFM parameters"
    afm_params_file = write_afm_params(params=afm_params).result

    ljff = shelljob(
        command="ppafm-generate-ljff",
        nodes={
            "geometry": geometry_file,
            "parameters": afm_params_file,
        },
        arguments=[
            "-i",
            "geo.xyz",
            "-f",
            "npy",
        ],
        outputs=["FFLJ.npz"],
    )

    scan_nodes = {
        "parameters": afm_params_file,
        "ljff_data": ljff.FFLJ_npz,
    }

    metadata = {
        "options": {
            "use_symlinks": True,
        }
    }

    if case != AfmCase.EMPIRICAL.name:
        if not relax:
            assert dft_params, "Missing DFT parameters"
            geom_dft_params = dft_params.get("structure", {})
            scf_params = geom_dft_params.get("base", {})
            assert scf_params, "Missing structure base SCF parameters"
            scf_params["pw.structure"] = structure
            scf_params["pw"]["parameters"]["CONTROL"]["calculation"] = "scf"
            dft_job = ScfJob(**scf_params)

        assert pp_params, "Missing post-processing parameters"
        hartree_params = pp_params.get("hartree", {})
        assert hartree_params, "Missing Hartree parameters"
        hartree_task = PpJob(
            parent_folder=dft_job.remote_folder,
            **hartree_params,
        )

        if case == AfmCase.HARTREE.name:
            elff = shelljob(
                command="ppafm-generate-elff",
                metadata=metadata,
                nodes={
                    "parameters": afm_params_file,
                    "ljff_data": ljff.FFLJ_npz,
                    "hartree_data": hartree_task.remote_folder,
                },
                filenames={
                    "hartree_data": "hartree",
                },
                arguments=[
                    "-i",
                    "hartree/aiida.fileout",
                    "-F",
                    "cube",
                    "-f",
                    "npy",
                ],
                outputs=["FFel.npz"],
            )

            scan_nodes["elff_data"] = elff.FFel_npz

        # Experimental feature, not fully tested
        elif case == AfmCase.HARTREE_RHO.name:
            charge_params = pp_params.get("charge", {})
            assert charge_params, "Missing charge density parameters"
            rho_job = PpJob(
                parent_folder=dft_job.remote_folder,
                **charge_params,
            )

            assert tip, "Missing tip structure"
            tip_scf_params = dft_params.get("tip", {})
            assert tip_scf_params, "Missing tip DFT parameters"
            tip_scf_params["pw.structure"] = tip
            tip_dft_job = ScfJob(**tip_scf_params)

            tip_rho_job = PpJob(
                parent_folder=tip_dft_job.remote_folder,
                **charge_params,
            )

            conv_rho = shelljob(
                command="ppafm-conv-rho",
                nodes={
                    "geom_density": rho_job.remote_folder,
                    "tip_density": tip_rho_job.remote_folder,
                },
                filenames={
                    "geom_density": "structure",
                    "tip_density": "tip",
                },
                arguments=[
                    "-s",
                    "structure/aiida.fileout",
                    "-t",
                    "tip/aiida.fileout",
                    "-B",
                    "1.0",
                    "-E",
                ],
            )

            charge_elff = shelljob(
                command="ppafm-generate-elff",
                nodes={
                    "conv_rho_data": conv_rho.remote_folder,
                    "hartree_data": hartree_task.remote_folder,
                    "tip_density": tip_rho_job.remote_folder,
                },
                filenames={
                    "conv_rho_data": "conv_rho",
                    "hartree_data": "hartree",
                    "tip_density": "tip",
                },
                arguments=[
                    "-i",
                    "hartree/aiida.fileout",
                    "-tip-dens",
                    "tip/aiida.fileout",
                    "--Rcode",
                    "0.7",
                    "-E",
                    "--doDensity",
                ],
                outputs=["FFel.npz"],
            )

            dftd3 = shelljob(
                command="ppafm-generate-dftd3",
                nodes={
                    "hartree_data": hartree_task.remote_folder,
                },
                filenames={
                    "hartree_data": "hartree",
                },
                arguments=[
                    "-i",
                    "hartree/aiida.fileout",
                    "--df_name",
                    "PBE",
                ],
            )

            elff = shelljob(
                command="ppafm-generate-elff",
                nodes={
                    "hartree_data": hartree_task.remote_folder,
                    "charge_elff_data": charge_elff.FFel_npz,
                },
                filenames={
                    "hartree_data": "hartree",
                },
                arguments=[
                    "-i",
                    "hartree/aiida.fileout",
                    "-f",
                    "npy",
                ],
                outputs=["FFel.npz"],
            )

        else:
            raise ValueError(f"Unsupported case: {case}")

    scan = shelljob(
        command="ppafm-relaxed-scan",
        metadata=metadata,
        nodes=scan_nodes,
        arguments=[
            "-f",
            "npy",
        ],
        outputs=["Q0.00K0.35"],
    )

    results = shelljob(
        command="ppafm-plot-results",
        metadata=metadata,
        nodes={
            "parameters": afm_params_file,
            "scan_dir": scan.Q0_00K0_35,
        },
        filenames={
            "scan_dir": "Q0.00K0.35",
        },
        arguments=[
            "--df",
            "--cbar",
            "--save_df",
            "-f",
            "npy",
        ],
        outputs=["Q0.00K0.35"],
    )

    return results


In [ ]:
from aiida_pseudo.groups.family.pseudo import PseudoPotentialFamily

structure = orm.StructureData()
structure.set_cell(
    [
        [14.9412827110, 0.0, 0.0],
        [0.0, 14.5091262213, 0.0],
        [0.0, 0.0, 10.0820001747],
    ]
)
structure.set_pbc((False, False, False))
structure.append_atom(symbols="C", position=(8.2766055186, 6.1177492411, 5.0595246063))
structure.append_atom(symbols="C", position=(8.8582363932, 7.3839854920, 5.0363959612))
structure.append_atom(symbols="C", position=(8.0523006568, 8.5208073256, 5.0178902940))
structure.append_atom(symbols="C", position=(6.6646772714, 8.3913773689, 5.0224782237))
structure.append_atom(symbols="C", position=(6.0830463468, 7.1251409580, 5.0456056988))
structure.append_atom(symbols="C", position=(6.8889822232, 5.9883191444, 5.0641114560))
structure.append_atom(symbols="H", position=(8.9093056269, 5.2253257080, 5.0739448106))
structure.append_atom(symbols="H", position=(9.9475591624, 7.4856249613, 5.0328185108))
structure.append_atom(symbols="H", position=(8.5089059407, 9.5148483491, 4.9998780126))
structure.append_atom(symbols="H", position=(6.0319773431, 9.2838011020, 5.0080584394))
structure.append_atom(symbols="H", position=(4.9937235476, 7.0235011287, 5.0491811392))
structure.append_atom(symbols="H", position=(6.4323774593, 4.9942778709, 5.0821221574))

tip = orm.StructureData()
tip.set_cell(
    [
        [20.0, 0.0, 0.0],
        [0.0, 20.0, 0.0],
        [0.0, 0.0, 20.0],
    ]
)
tip.set_pbc((False, False, False))
tip.append_atom(symbols="C", position=(0.0, 0.0, 1.15))
tip.append_atom(symbols="O", position=(0.0, 0.0, 0.0))

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])

pseudo_family = t.cast(PseudoPotentialFamily, orm.load_group(4))
C_pp = pseudo_family.get_pseudo("C")
H_pp = pseudo_family.get_pseudo("H")
O_pp = pseudo_family.get_pseudo("O")

In [ ]:
afm_params = {
    "PBC": False,
    "tip": "s",
    "klat": 0.3490127886809,
    "krad": 21.913190531846,
    "gridA": [14.9412827110, 0.0000000000, 0.0000000000],
    "gridB": [0.0000000000, 14.5091262213, 0.0000000000],
    "gridC": [0.0000000000, 0.0000000000, 10.0820001747],
    "sigma": 0.7,
    "charge": 0.0,
    "r0Probe": [0.0, 0.0, 2.97],
    "scanMax": [14.9412827110, 14.5091262213, 11],
    "scanMin": [0.0, 0.0, 8],
    "scanStep": [0.1, 0.1, 0.1],
    "Amplitude": 1.4,
    "probeType": "O",
    "f0Cantilever": 22352.5,
    "gridN": [-1, -1, -1],
}

In [ ]:
pw_metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
        },
        "max_wallclock_seconds": 43200,
    }
}

pw_code = orm.load_code("pw-7.4@localhost")

dft_params = {
    "structure": {
        "base": {
            "metadata": {},
            "pw": {
                "code": pw_code,
                "metadata": pw_metadata,
                "pseudos": {
                    "C": C_pp,
                    "H": H_pp,
                },
                "parameters": {
                    "CONTROL": {
                        "calculation": "relax",
                        "forc_conv_thr": 0.001,
                        "tprnfor": True,
                        "tstress": True,
                        "etot_conv_thr": 0.0002,
                        "nstep": 50,
                    },
                    "SYSTEM": {
                        "nosym": False,
                        "occupations": "fixed",
                        "ecutrho": 240.0,
                        "ecutwfc": 30.0,
                        "tot_charge": 0.0,
                        "vdw_corr": "none",
                    },
                    "ELECTRONS": {
                        "electron_maxstep": 80,
                        "mixing_beta": 0.4,
                        "conv_thr": 8e-10,
                    },
                },
            },
            "kpoints": kpoints,
            "kpoints_force_parity": False,
            "max_iterations": 5,
        },
        "max_meta_convergence_iterations": 5,
        "meta_convergence": True,
        "volume_convergence": 0.05,
    },
    "tip": {
        "metadata": {},
        "pw": {
            "code": pw_code,
            "metadata": pw_metadata,
            "pseudos": {
                "C": C_pp,
                "O": O_pp,
            },
            "parameters": {
                "CONTROL": {
                    "calculation": "scf",
                    "forc_conv_thr": 0.001,
                    "tprnfor": True,
                    "tstress": True,
                    "etot_conv_thr": 0.0002,
                    "nstep": 50,
                },
                "SYSTEM": {
                    "nosym": False,
                    "occupations": "fixed",
                    "ecutrho": 240.0,
                    "ecutwfc": 30.0,
                    "tot_charge": 0.0,
                    "vdw_corr": "none",
                },
                "ELECTRONS": {
                    "electron_maxstep": 80,
                    "mixing_beta": 0.4,
                    "conv_thr": 8e-10,
                },
            },
        },
        "kpoints": kpoints,
        "kpoints_force_parity": False,
        "max_iterations": 5,
    },
}

In [ ]:
pp_metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
        },
        "max_wallclock_seconds": 43200,
    }
}

pp_code = orm.load_code("pp-7.4@localhost")

pp_params = {
    "hartree": {
        "metadata": pp_metadata,
        "code": pp_code,
        "parameters": {
            "INPUTPP": {
                "plot_num": 11,
            },
            "PLOT": {
                "iflag": 3,
            },
        },
    },
    "charge": {
        "metadata": pp_metadata,
        "code": pp_code,
        "parameters": {
            "INPUTPP": {
                "plot_num": 0,
            },
            "PLOT": {
                "iflag": 3,
            },
        },
    },
}


In [ ]:
wg = AfmWorkflow.build(
    case=AfmCase.EMPIRICAL.name,
    structure=structure,
    afm_params=afm_params,
    relax=False,
    dft_params=dft_params,
    pp_params=pp_params,
    tip=tip,
)

In [ ]:
wg

In [ ]:
wg.submit()

In [ ]:
node = orm.load_node(wg.pk)

In [ ]:
import base64

from IPython.display import HTML

fd: orm.FolderData = node.outputs.Q0_00K0_35

imgs = []

png_folder = "Amp1.40"

for obj in fd.list_objects(png_folder):
    if obj.name.endswith(".png"):
        with fd.open(f"{png_folder}/{obj.name}", "rb") as handle:
            data = handle.read()
            data64 = base64.b64encode(data).decode("utf-8")
            imgs.append(f"""
                <img
                    src="data:image/png;base64,{data64}"
                    style="max-width:150px; margin:5px;"
                />
            """)

HTML("".join(imgs))